<a href="https://colab.research.google.com/github/guitar79/OA-2018/blob/master/08_3_Read_fits_mono_16bit_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Read fits file**


* 이 자료는16 bit Fits file

* python이 처음이라면 [Python Basic](https://colab.research.google.com/drive/1PCOoDIKypPVX9KTItMOht1cl96cPmeR_?authuser=1#scrollTo=3g6o04iLM0AF), [Python packages](https://colab.research.google.com/drive/1-1wx2VPEyNe11bmgpSpwdQgrJASCAqdH?authuser=1)를 먼저 학습하기를 권한다.

* package를 쉽게 설치하기 위해 Anaconda 사용을 권장한다.

경기과학고등학교 관측천문학 강좌를 위해 만들었으며  <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">크리에이티브 커먼즈 저작자표시-비영리-동일조건변경허락 4.0 국제 라이선스</a>에 따라 이용할 수 있음.

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="크리에이티브 커먼즈 라이선스" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a>

#Download data

Download fits files

* "M80"을 검색하면 많은 데이타가 보일 것이다.
* "Detector" column : "WFPC2" 를 입력하고 "Enter"
* "NExposures" column : 6
* "PropID" column : "11233"
* "VisitNum" = 06 

In [0]:
!pip install astropy
import numpy as np
import cv2
from astropy.io import fits
import os

# file name
f_name = '20181012.M42'

hdu = fits.open(f_name)
data = hdu[0].data
cv2_arrray = np.array(data, dtype=np.uint16)

print(cv2_arrray)  

    100% |████████████████████████████████| 6.0MB 6.6MB/s 


# Processing data

HR diagram 을 그리기 위해 필요한 자료는 다음과 같다.


*  $\rm{Color~Index} = M_{F450} - M_{F814}$
*  $\rm{Magnitude} = M_{F450}$

다운받은 자료에서 별의 ID (5th colomn), 별의 Magnitude (6th column)를 이용하면 된다.



In [0]:
print (F450)
print (F450[:,4])

여기서 해결해야 할 문제는 두 파일 (F450, F814)에서 데이터 갯수가 서로 다르다는 것이다. 

이를 해결하려면 별의 ID를 비교하는 과정이 필요할텐데 다음과 같은 bool data type을 이용하면 해결할 수 있을 것이다.

In [0]:
import numpy as np

num1 = np.arange(0, 11, 2) # generate numpy array (0, 11) with interval 2

print("\nnum1's elements:")
print(num1)

# MASKING
tf   = np.array([False, True, False, True, True, False])

print (tf)

print("\nnum1's selected elements:")
print(num1[tf])


먼저 F450과 F814 의 데이터 갯수를 하나나하 loop 문으로 비교해야 할 것이다.

In [0]:
print (len(F450))
print (len(F814))

F450에 있는 별의 ID와 일치하는 별이 F814에 있는지 찾아서 색지수를 구하고 일치하는 자료의 bool 자료를 true로 저장하자.

In [0]:
# get CI value and masking 
mask  = np.zeros(len(F450)).astype(bool)
color = np.zeros(len(F450))

# Get color indices of the "same" stars in two filter images
for i in range(0, len(F450)):
    ID450 = F450[i,4]
    for j in range(0, len(F814)):
        ID814 = F814[j,4]
        if ID450 == ID814:
            mask[i]  = True
            color[i] = F450[i,5] - F814[j,5]
            break # Once the matched item is found from F814, go to next star of F450.
print (len(F450[mask]))
print (len(color[mask]))

# Chart plot

matplotlib를 이용하면 아래와 같은 코드로 HR diagram 을 그릴 수 있을 것이다.

In [0]:
import numpy as np
import matplotlib.pyplot as plt

x = np.arange(0,6,0.1)
y = np.random.rand(60)
CI = x 
Mag = y*20
print("CI\n", CI)
print("Mag\n", Mag)

#plot CI - Mag
plt.plot(CI, Mag, 'o', ms=5, alpha=1)
plt.grid()
plt.title('H-R Diagram')
plt.xlabel('Color Index')
plt.ylabel('Magnitude')
plt.gca().invert_yaxis()

#show the Diagram
plt.show()

$\rm{Color~Index}$와 $\rm{Magnitude}$ 값을 indexing 하는 코드는 각각 color[mask], F450[mask,5]이다. 

In [0]:
print(color[mask])
print(F450[mask,5])

HR diagram을 그려보자.

In [0]:
# Get current size
fig_size = plt.rcParams["figure.figsize"]
 
# Prints: [8.0, 6.0]
print ("Current size:", fig_size)

# Set figure width to 12 and height to 9
fig_size[0] = 10
fig_size[1] = 8
plt.rcParams["figure.figsize"] = fig_size


plt.plot(color[mask], F450[mask,5], 'o', ms=2, alpha=0.2)
plt.title('H-R Diagram of M80', fontsize=20)
plt.xlabel('Color Index (F450W - F814W)', fontsize=18)
plt.ylabel('Magnitude (F450W)', fontsize=18)
plt.gca().invert_yaxis()

plt.grid()


#show the Diagram
plt.show()

plt.savefig('H-R Diagram of M80.png', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)

정리 차원에서 최종 코드와 HST 자료를 첨부한다.

* [full code](https://github.com/guitar79/OA-2018/blob/master/08-align_and_combine_fits_mono_16bit.py)